In [469]:
from dotenv import load_dotenv
load_dotenv()
from google.cloud.firestore import Client as FirestoreClient
from chat_history_service import ChatHistoryService
from llm_request_service import LLMRequestService
from typing import List, Dict, Any
import pandas as pd
import os
import json
import openpyxl
import xlsxwriter
from datetime import datetime
from zoneinfo import ZoneInfo 
from textwrap import fill

## Define Functions

In [470]:
def get_chat_history_from_firestore(session_ids: list[str]) -> dict[str, list[dict]]:
    """
    Fetch chat histories from Firestore for given session IDs.
    
    Args:
        session_ids (list[str]): The list of session IDs to fetch chat histories for.
    
    Returns:
        dict[str, list[dict]]: Dictionary of session IDs and their respective chat histories.
    """
    chat_history_service = ChatHistoryService()
    chat_histories = chat_history_service.get_chat_histories_by_ids(session_ids)
    
    for session_id, history in chat_histories.items():
        chat_histories[session_id] = sorted(history, key=lambda x: x.get("create_time", ""))
    
    print(chat_histories)
    return chat_histories

In [471]:
def get_specific_data_from_history(
    history: List[Dict[str, Any]],
) -> List[Dict[str, Any]]:
    """
    Retrieves specific data from the entire conversation history, maintaining the conversation flow.

    Args:
        history (List[Dict[str, Any]]): List of dictionaries containing message history.
    s
    Returns:
        List[Dict[str, Any]]: List of dictionaries, each containing user message, bot responses, and button label for each turn in the conversation.
    """
    # Sort the history by create_time
    sorted_history = sorted(history, key=lambda x: x.get("create_time"))

    # Initialize list to store the conversation turns
    conversation = []

    # Iterate through the history
    for message in sorted_history:
        turn = {}

        # Preserve the order of events and skip None values
        if "user_msg" in message and message["user_msg"] is not None:
            turn["user_msg"] = message["user_msg"]
        if "button_label" in message and message["button_label"] is not None:
            turn["button_label"] = message["button_label"]
        if "bot_response" in message and message["bot_response"] is not None:
            bot_response = message["bot_response"]
            if isinstance(bot_response, list):
                bot_messages = []
                for response in bot_response:
                    if isinstance(response, dict) and "message" in response and response["message"] is not None:
                        bot_messages.append({"message": response["message"]})
                if bot_messages:
                    turn["bot_response"] = bot_messages

        # Add the turn to the conversation if it's not empty
        if turn:
            conversation.append(turn)

    return conversation

In [472]:
def get_specific_chat_data(chat_history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Selects specific data from the chat history.
    Args:
        chat_history (List[Dict[str, Any]]): The chat history to process.
    Returns:
        List[Dict[str, Any]]: Selected specific data from the chat history.
    """
    specific_chat_history = get_specific_data_from_history(chat_history)
    return specific_chat_history

In [473]:
def get_specific_chat_data_for_multiple_sessions(
    chat_histories: Dict[str, List[Dict[str, Any]]]
) -> Dict[str, List[Dict[str, Any]]]:
    """
    Fetches and processes chat histories for multiple sessions.
    Args:
        chat_histories (Dict[str, List[Dict[str, Any]]]): Dictionary of session IDs and their corresponding chat histories.
    Returns:
        Dict[str, List[Dict[str, Any]]]: Dictionary of session IDs and their processed specific chat data.
    """
    processed_data = {}
    for session_id, history in chat_histories.items():
        processed_data[session_id] = get_specific_data_from_history(history)
    return processed_data

In [474]:
def create_pretty_dataframe(processed_data):
    rows = []
    for session_id, conversation in processed_data.items():
        dialogue = ""
        for turn in conversation:
            # Check for button label first
            if 'button_label' in turn and turn['button_label'] is not None and turn['button_label'] != "":
                dialogue += f"User button: {turn['button_label']}\n\n"
            
            # Then check for user message
            if 'user_msg' in turn and turn['user_msg'] is not None and turn['user_msg'] != "":
                dialogue += f"User: {turn['user_msg']}\n\n"
            
            # Finally, add bot responses
            if 'bot_response' in turn and turn['bot_response'] is not None:
                for response in turn['bot_response']:
                    if isinstance(response, dict) and 'message' in response and response['message'] is not None:
                        dialogue += f"Bot: {response['message']}\n"
            
            dialogue += "\n"

        rows.append({'Session ID': session_id, 'Dialogue': dialogue.strip()})
    
    dialogue_df = pd.DataFrame(rows)
    return dialogue_df

In [475]:
def call_llm_service_summarize(conversations: Dict[str, str], prompt: str = "") -> Dict[str, str]:
    """
    Call the LLM service to summarize multiple chat histories.
    Args:
        conversations (Dict[str, str]): A dictionary of chat histories to summarize, 
                                        where the key is the session ID and the value is the conversation.
        prompt (str): The prompt to send to the LLM service.
                      If not provided, the default prompt will be fetched from Firestore.
    Returns:
        Dict[str, str]: A dictionary of summaries, where the key is the session ID and the value is the summary.
    """
    llm_request_service = LLMRequestService()
    chat_history_summaries = llm_request_service.summarize_conversations(
        conversations, prompt
    )
    return chat_history_summaries

In [476]:
def format_summary(summary: str) -> str:
    # Split the summary into sentences
    sentences = summary.split(". ")
    # Join the sentences with line breaks
    formatted_summary = ".\n".join(sentences)
    return f"Summary:\n{formatted_summary}"


def count_output(text):
    """
    Count characters, words, and lines in the given text.
    """
    char_count = len(text)
    word_count = len(text.split())
    return char_count, word_count

In [477]:
def create_conversation_summary_dataframe(processed_data, summaries):
    rows = []
    for session_id, conversation in processed_data.items():
        # Format conversation
        dialogue = ""
        for turn in conversation:
            if 'button_label' in turn and turn['button_label'] is not None and turn['button_label'] != "":
                dialogue += f"User button: {turn['button_label']}\n\n"
            if 'user_msg' in turn and turn['user_msg'] is not None and turn['user_msg'] != "":
                dialogue += f"User: {turn['user_msg']}\n\n"
            if 'bot_response' in turn and turn['bot_response'] is not None:
                for response in turn['bot_response']:
                    if isinstance(response, dict) and 'message' in response and response['message'] is not None:
                        dialogue += f"Bot: {response['message']}\n"
            dialogue += "\n"
        
        # Get summary and character count
        summary = summaries.get(session_id, "")
        summary_char_count = len(summary)
        
        # Create row
        row = {
            "Session ID": session_id,
            "Conversation": dialogue.strip(),
            "Summary": summary,
            "Summary Character Count": summary_char_count
        }
        rows.append(row)
    
    return pd.DataFrame(rows)

In [478]:
def call_llm_service_queue(
    summaries: Dict[str, str],
    customer_comment: str = "",
    service: str = "",
    telesales: str = "",
    techniek: str = "",
    activatie: str = "",
    tag_prompt: str = "",
) -> Dict[str, str]:
    """
    Call the LLM service to tag the call queue for multiple sessions.
    Args:
        summaries (Dict[str, str]): Dictionary of session IDs and their summaries.
        customer_comment (str): The single customer comment applicable to all sessions.
        tag_prompt (str): The prompt to send to the LLM service for tagging.
    Returns:
        Dict[str, str]: Dictionary of session IDs and their tagged call queues.
    """
    llm_request_service = LLMRequestService()
    tagged_queues = {}
    
    for session_id, summary in summaries.items():
        tagged_queue = llm_request_service.tag_callback(
            session_id, summary, customer_comment, service, telesales, techniek, activatie, tag_prompt
        )
        tagged_queues[session_id] = tagged_queue
    
    return tagged_queues

## Gather Conversation

### Get conversation from session_id in Development

In [479]:
# session_id = "00001cac-a1fe-493d-8c04-813527671906" # Replace with other session ID
# chat_history = get_chat_histories_from_firestore(session_id)
# chat_history

# conversation = get_specific_chat_data(chat_history)
# conversation

In [480]:
session_ids = [
    "00001a3e-6022-4d94-a506-143856c60b0e",
    "00000bdf-17cf-429e-aba9-bf5cfe4f9ba6",
    "00001339-f528-42df-b90f-d07ec400c96d",
    "00001e6b-25cb-429c-aff6-06320956a52c",
    "00001ede-c4a9-46f3-afea-099d42067d21",
    "000023e2-134e-4562-a98c-193de0ead3f7",
    "00003010-482e-43d0-ad5d-8518a3623a70",
    "00003633-0f8a-4197-b2f0-9f412318bd11",
    "00003bec-136b-430d-a53e-c30718bc6839",
    "00003ee6-c922-4d56-87a0-bbf79f9bdc16",
    "000040f6-fab4-4090-947d-59134706016d",
    "000043be-dc9f-4f88-9cc2-fd4857540773",
    "00004f04-4810-4b9f-87e9-6712d4d1d6aa",
    "00005426-124f-4cfb-b8e1-2539803ca839",
    "000071f0-e136-4e58-8b6c-b46f1be9908a",
    "0000adf4-e7f4-4824-9146-641f7e3c4fe8",
    "0000b2f6-2244-4995-9e08-bf19def08bfd",
    "0000c8d4-d6e1-427d-8e78-604e6f4847c2",
    "0000e2b3-0f4c-45b4-80f6-0fa9742230c4",
    "0000ef07-14a7-4b4f-a6e4-958ab541843e"
]  
chat_histories = get_chat_history_from_firestore(session_ids)
chat_histories

{'00001a3e-6022-4d94-a506-143856c60b0e': [{'sender': 'bot', 'response_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 460779, tzinfo=datetime.timezone.utc), 'intent_id': 'MT_SYSTEM_PRODUCTSELECT', 'user_msg': None, 'bot_response': [{'message_en': "Hi, I'm chatbot Izzi from Odido.", 'type': 'text', 'message': 'Hallo, ik ben chatbot Izzi van Odido.'}, {'message_en': 'Which product is your question about?', 'language': 'nl', 'buttons': [{'label': 'Mobiel', 'event': 'MT_SYSTEM_PRODUCTSELECT-choice;mobiel', 'label_en': 'Mobile'}, {'label': 'Internet + TV', 'event': 'MT_SYSTEM_PRODUCTSELECT-choice;thuis', 'label_en': 'Internet + TV'}], 'type': 'button-list', 'escapeOption': False, 'message': 'Over welk product wil je iets vragen?'}], 'button_label': None, 'create_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 483898, tzinfo=datetime.timezone.utc)}, {'sender': 'bot', 'response_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 30, 465481, tzinfo=datetime.timezone.utc), 'intent_id'

{'00001a3e-6022-4d94-a506-143856c60b0e': [{'sender': 'bot',
   'response_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 460779, tzinfo=datetime.timezone.utc),
   'intent_id': 'MT_SYSTEM_PRODUCTSELECT',
   'user_msg': None,
   'bot_response': [{'message_en': "Hi, I'm chatbot Izzi from Odido.",
     'type': 'text',
     'message': 'Hallo, ik ben chatbot Izzi van Odido.'},
    {'message_en': 'Which product is your question about?',
     'language': 'nl',
     'buttons': [{'label': 'Mobiel',
       'event': 'MT_SYSTEM_PRODUCTSELECT-choice;mobiel',
       'label_en': 'Mobile'},
      {'label': 'Internet + TV',
       'event': 'MT_SYSTEM_PRODUCTSELECT-choice;thuis',
       'label_en': 'Internet + TV'}],
     'type': 'button-list',
     'escapeOption': False,
     'message': 'Over welk product wil je iets vragen?'}],
   'button_label': None,
   'create_time': DatetimeWithNanoseconds(2024, 5, 1, 7, 41, 23, 483898, tzinfo=datetime.timezone.utc)},
  {'sender': 'bot',
   'response_time': D

In [481]:
# Process the histories to get specific data for multiple sessions
processed_data = get_specific_chat_data_for_multiple_sessions(chat_histories)

processed_data

{'00001a3e-6022-4d94-a506-143856c60b0e': [{'bot_response': [{'message': 'Hallo, ik ben chatbot Izzi van Odido.'},
    {'message': 'Over welk product wil je iets vragen?'}]},
  {'button_label': 'Internet + TV',
   'bot_response': [{'message': 'Komen we er samen niet uit, dan zijn onze adviseurs er voor je.'},
    {'message': 'Wat kan ik voor je doen?'}]},
  {'button_label': 'Bestelstatus',
   'bot_response': [{'message': 'Ik vertel je graag meer over de status van je bestelling.'},
    {'message': 'Heb je een mail ontvangen met de bevestiging van je bestelling?'}]},
  {'button_label': 'Andere vraag',
   'bot_response': [{'message': 'Een andere vraag kan natuurlijk ook. Hoe kan ik je helpen?'}]},
  {'user_msg': 'Adviseur spreken',
   'bot_response': [{'message': 'Ik ga voor je kijken of een adviseur je kan helpen.'},
    {'message': "Kies 'Verkoop' voor hulp bij bestellen of verlengen van een abonnement en/of telefoon. Kies 'Service' voor andere vragen."}]},
  {'button_label': 'Verkoop',

In [482]:
# Process the histories to get specific data for multiple sessions
processed_data = get_specific_chat_data_for_multiple_sessions(chat_histories)

# Create the pretty DataFrame
pretty_df = create_pretty_dataframe(processed_data)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)  # Adjust this value as needed

# Function to wrap text
def wrap_text(text, width=100):
    return '\n'.join(fill(line, width) for line in text.split('\n'))

# Apply text wrapping to the Dialogue column
pretty_df['Dialogue'] = pretty_df['Dialogue'].apply(wrap_text)

# Display the DataFrame
for index, row in pretty_df.iterrows():
    print(f"Session ID: {row['Session ID']}")
    print(row['Dialogue'])
    print("-" * 100)  # Separator
    print()

Session ID: 00001a3e-6022-4d94-a506-143856c60b0e
Bot: Hallo, ik ben chatbot Izzi van Odido.
Bot: Over welk product wil je iets vragen?

User button: Internet + TV

Bot: Komen we er samen niet uit, dan zijn onze adviseurs er voor je.
Bot: Wat kan ik voor je doen?

User button: Bestelstatus

Bot: Ik vertel je graag meer over de status van je bestelling.
Bot: Heb je een mail ontvangen met de bevestiging van je bestelling?

User button: Andere vraag

Bot: Een andere vraag kan natuurlijk ook. Hoe kan ik je helpen?

User: Adviseur spreken

Bot: Ik ga voor je kijken of een adviseur je kan helpen.
Bot: Kies 'Verkoop' voor hulp bij bestellen of verlengen van een abonnement en/of telefoon. Kies
'Service' voor andere vragen.

User button: Verkoop

Bot: Ik kan je op dit moment helaas niet doorverbinden met een adviseur.
Bot: Je kan onze adviseurs bellen op 0800-0092. Op maandag t/m vrijdag zijn we bereikbaar van 8-20
uur, op zaterdag van 9-17.30 uur, op zondag van 12-17.30 uur, op feestdagen zijn 

## Manage Summary Prompt

In [483]:
prompt = """
Analyseer deze chat {text} en beschrijf kort waarom de (potentiële) klant contact opneemt. 
Focus alleen op de klantinput in "user_msg" en "button_label". Negeer begroetingen, afscheid, handovers naar verkoop/service medewerkers, smalltalk, emoties en beledigingen.
Benoem niet de productselectie aan het begin van het gesprek.
Benoem niet dat de klant een medewerker/adviseur wil spreken of een terugbelverzoek aanvraagt.
Houd je omschrijving kort en bondig. Gebruik maximaal 100 tekens. Schrijf in het Nederlands. Begin met "Je...".
"""

## Generate Summaries

In [484]:
# summary = call_llm_service_summarize(conversations, prompt)
# formatted_output = format_summary(summary)
# print(formatted_output)
# print()

# # Count the output
# char_count, word_count = count_output(formatted_output)
# print(f"Characters: {char_count}")
# print(f"Words: {word_count}")

In [485]:
# Assuming you have chat_histories
conversations = get_specific_chat_data_for_multiple_sessions(chat_histories)

# Call the summarization function
summaries = call_llm_service_summarize(conversations, prompt)

# Print summaries
for session_id, summary in summaries.items():
    formatted_summary = format_summary(summary)
    print(f"Session ID: {session_id}")
    print(formatted_summary)
    
    char_count, word_count = count_output(formatted_summary)
    print(f"Characters: {char_count}")
    print(f"Words: {word_count}")
    print()

Session ID: 00001a3e-6022-4d94-a506-143856c60b0e
Summary:
Je vraagt naar de status van je bestelling en hebt andere vragen over producten of diensten.
Characters: 101
Words: 17

Session ID: 00000bdf-17cf-429e-aba9-bf5cfe4f9ba6
Summary:
Je hebt geen wifi meer en zoekt naar een oplossing voor je internetverbinding.
Characters: 87
Words: 14

Session ID: 00001339-f528-42df-b90f-d07ec400c96d
Summary:
Je hebt een vraag over het aanvullen van data in zone 2, maar kunt dit niet doen.
Characters: 90
Words: 18

Session ID: 00001e6b-25cb-429c-aff6-06320956a52c
Summary:
Je hebt een vraag over een product van Odido.
Characters: 54
Words: 10

Session ID: 00001ede-c4a9-46f3-afea-099d42067d21
Summary:
Je wilt het adres wijzigen in de systemen van Odido.
Characters: 61
Words: 11

Session ID: 000023e2-134e-4562-a98c-193de0ead3f7
Summary:
Je vraagt naar een oplossing voor een kapotte simkaart en vergeten inloggegevens.
Characters: 90
Words: 13

Session ID: 00003010-482e-43d0-ad5d-8518a3623a70
Summary:
Je

In [486]:
# Assuming you have processed_data and summaries
conversation_summary_df = create_conversation_summary_dataframe(processed_data, summaries)

# Display the DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
print(conversation_summary_df)

# # Create the output folder if it doesn't exist
# output_folder = "output"
# os.makedirs(output_folder, exist_ok=True)

# # Export to Excel
# excel_filename = os.path.join(output_folder, "conversation_summaries.xlsx")
# conversation_summary_df.to_excel(excel_filename, index=False)
# print(f"Data exported to {excel_filename}")

                              Session ID                                                                                         Conversation                                                                                       Summary  Summary Character Count
0   00001a3e-6022-4d94-a506-143856c60b0e  Bot: Hallo, ik ben chatbot Izzi van Odido.\nBot: Over welk product wil je iets vragen?\n\nUser b...  Je vraagt naar de status van je bestelling en hebt andere vragen over producten of diensten.                       92
1   00000bdf-17cf-429e-aba9-bf5cfe4f9ba6  User: Ik heb geen wifi meer\n\nBot: Ik begrijp dat je een vraag hebt over Internet (+ TV + Vast ...                Je hebt geen wifi meer en zoekt naar een oplossing voor je internetverbinding.                       78
2   00001339-f528-42df-b90f-d07ec400c96d  Bot: Goedemiddag Sonja, ik ben chatbot Izzi. Welkom bij Odido. Je kan bij mij terecht met je vra...             Je hebt een vraag over het aanvullen van data in zone 2, maar k

## Update the prompt in Firestore Database (code uses the uploaded one)

In [487]:
# firestore_client = FirestoreClient()
# collection = firestore_client.collection("llm-prompts")
# document = collection.document("fulfillment-webhook")
# document.update({"summarize-chat-conversation": prompt})

# Manage Parameters and Call Tag Queue Prompt

In [488]:
# service = """
# Voorbeelden: 
# - Vragen over facturen en betalingen (bijvoorbeeld uitleg over de factuur, incassodatum, betalingstermijn, betaalprobleem)
# - Vragen over betaalproblemen en betalingsregelingen
# - Vragen over extra kosten of inflatiecorrectie
# - Assistentie bij pin/puk-codes
# - Vragen over huidige abonnement zoals abonnementstatus, startdatum, einddatum, abonnementstype
# - Hulp met of vragen over de Mijn Odido website omgeving of de Odido app
# - Wachtwoord en inlogproblemen
# - Hulp bij abonnement koppelen
# - Wijzigen van persoonlijke of betaalgegevens
# - Verzoeken voor contractsovername
# - Vragen over sim- en esimkaarten zoals aanvragen van een vervangende simkaart
# - Assistentie bij nummerportering
# - Vragen rondom verhuizing
# - Informatie over dienstgebruik, tarieven, datalimieten en roaming
# - Aanpassen van extra's, bundels, mb's, belminuten, tv-zenders en meer
# - Informatie over lopende bestellingen en leveringen zoals bestelstatus en retouren
# - Instellingen zoals voicemail, 5G uitzetten, blokkades en meer

# """

# telesales = """
# Voorbeelden:
# - Afsluiten van een nieuw abonnement, dienst of product
# - Verlengen van een bestaand abonnement, dienst of product
# - Opzeggen van een abonnement, dienst of product (zowel tijdens als na de contractperiode)
# """

# techniek = """
# Voorbeelden:
# - Problemen met thuisinternet (geen verbinding of slechte kwaliteit)
# - Problemen met mobiel internet (geen verbinding of slechte bereik)
# - Slechte TV-ontvangst of niet-werkende TV-kastjes
# - Router-gerelateerde problemen
# - Problemen met telefonisch bereik
# - Niet-beschikbare TV-zenders
# - Algemene storingen met diensten
# """

# activatie = """
# Voorbeelden:
# - Vragen over de planning van de monteur
# - Informatie over de aansluitdatum
# - Problemen na aansluiting (bijvoorbeeld wel aangesloten maar geen diensten)
# - Melden van niet-verschenen monteurs
# """


In [ ]:
service = """
Voorbeelden: 
- Hulp bij vragen over facturen en betalingen
- Hulp bij betaalproblemen en het treffen van betalingsregelingen
- Uitleg over extra kosten of inflatiecorrectie
- Hulp bij pin- en puk-code problemen
- Informatie over huidige abonnement
- Hulp bij gebruik van Mijn Odido en de app
- Hulp bij inlogproblemen 
- Hulp bij abonnement koppelen
- Wijzigen van persoonlijke of betaalgegevens
- Hulp bij contractsovername
- Hulp bij en aanvragen van (vervangende) sim- en esimkaarten
- Hulp bij nummerportering en de overstapservice
- Uitleg over dienstgebruik, tarieven en roaming
- Hulp bij en aanvragen van Klantvoordeel
- Informatie over bestellingen, leveringen en retourneren
- Wijzigen van instellingen (bijvoorbeeld voicemail, 5G, blokkades)
"""

telesales = """
Voorbeelden:
- Afsluiten van een nieuw abonnement of dienst
- Verlengen van bestaand abonnement of dienst
- Toevoegen van extra diensten of producten
- Bestellen van hardware of accessoires
- Wijzigen (upgraden/downgraden) van huidig abonnement, bundels of extra's
- Opzeggen van een abonnement, product of dienst
- Advies over verschillende abonnementsvormen
- Informatie over actuele aanbiedingen en promoties
"""

techniek = """
Voorbeelden:
- Hulp bij algemene storingen met diensten
- Hulp bij problemen met mobiel of vast internet (wifi)
- Hulp bij problemen met bellen of sms'en
- Hulp bij TV-ontvangstproblemen en niet beschikbare zenders
- Hulp bij problemen met of instellen van routers, modems of TV-ontvangers
"""

activatie = """
Voorbeelden:
- Informatie over bestaande monteursafspraak
- Plannen van een (monteur)afspraak voor installatie
- Melden van een niet-verschenen monteur
- Informatie over het aansluitproces, de aansluitdatum en -tijd
- Hulp bij problemen na recente installatie
- Activeren van diensten na installatie
"""

In [490]:
customer_comment = """ik wil contact met een mens"""

In [491]:
tag_prompt = """
Analyseer de contactreden van een (potentiële) klant van Odido, een telecombedrijf, en categoriseer deze in de juiste klantenservice queue.

Gegeven informatie:
- Primaire contactreden: {text}
- Aanvullende klantopmerking: {customer_comment}
Als de aanvullende klantopmerking afwijkt van de primaire contactreden, hecht dan meer waarde aan de klantopmerking mits dit een inhoudelijke toevoeging is.

Beschikbare queues:
1. Queue: Service
Omschrijving: Behandelt algemene klantvragen, waaronder facturering, administratieve handelingen, accountbeheer en informatie over diensten.
Voorbeelden: {service}
2. Queue: Telesales
Omschrijving: Verantwoordelijk voor alle verkoop-gerelateerde activiteiten, zoals het afsluiten van nieuwe abonnementen en diensten, advies over abonnementsvormen, verlengingen, en het verwerken van opzeggingen.
Voorbeelden: {telesales}
3. Queue: Techniek
Omschrijving: Lost technische problemen op gerelateerd aan slecht of geen bereik van internet thuis, TV, mobiel internetten, bellen of sms'en, evenals problemen met apparatuur.
Voorbeelden: {techniek}
4. Queue: Activatie
Omschrijving: Begeleidt nieuwe klanten gedurende het aansluitproces voor thuisdiensten (internet, TV, vaste telefonie), van planning tot en met de succesvolle activatie van diensten.
Voorbeelden: {activatie}

Instructies:
1. Analyseer de contactreden en eventuele aanvullende opmerking.
2. Analyseer de queues met de bijbehorende omschrijvingen en voorbeelden.
3. Geef een beknopte argumentatie (maximaal 100 tekens) waarom de klantvraag bij een bepaalde queue zou passen.
4. Kies op basis van deze argumentatie de meest geschikte queue. Bij twijfel kies dan altijd voor de queue "Service".

Negeer het als mensen om contact vragen of teruggebeld willen worden.

Vereiste output:
Genereer een JSON-object met de volgende structuur:
{{
    "queue": "<naam van de gekozen queue>",
    "reason": "<beknopte reden voor de gekozen queue>"
}}

Voeg hierbij nooit "```json" aan toe.

Zorg ervoor dat de output strikt voldoet aan dit JSON-formaat, zonder extra witruimte of aanvullende tekst.
"""

In [492]:
# tag_prompt = """
# Analyseer de contactreden van een (potentiële) klant van Odido, een telecombedrijf, en categoriseer deze in de juiste klantenservice queue.

# Gegeven informatie:
# - Primaire contactreden: {text}
# - Aanvullende klantopmerking: {customer_comment}
# Als de aanvullende klantopmerking afwijkt van de primaire contactreden, hecht dan meer waarde aan de klantopmerking mits dit een inhoudelijke toevoeging is.

# Beschikbare queues:
# 1. Queue: Service
# Omschrijving: Behandelt vragen over facturering, wijzigingen, en algemene informatie over diensten en accounts.
# Voorbeelden: {service}
# 2. Queue: Telesales
# Omschrijving: Handelt abonnementsaanvragen, nieuwe bestellingen, verlengingen en opzeggingen af.
# Voorbeelden: {telesales}
# 3. Queue: Techniek
# Omschrijving: Lost technische problemen op gerelateerd aan slecht of geen bereik van internet thuis, TV, mobiel internetten, bellen of smsen.
# Voorbeelden: {techniek}
# 4. Queue: Activatie
# Omschrijving: Begeleidt nieuwe klanten bij het aansluitproces van thuisinternet en/of TV.
# Voorbeelden: {activatie}

# Instructies:
# 1. Analyseer de contactreden en eventuele aanvullende opmerking.
# 2. Analyseer de queues met de bijbehorende omschrijvingen en voorbeelden.
# 3. Geef een beknopte argumentatie (maximaal 100 tekens) waarom de klantvraag bij een bepaalde queue zou passen.
# 4. Kies op basis van deze argumentatie de meest geschikte queue. Bij twijfel kies dan altijd voor de queue "Service".

# Vereiste output:
# Genereer een JSON-object met de volgende structuur:
# {{
#     "queue": "<naam van de gekozen queue>",
#     "reason": "<beknopte reden voor de gekozen queue>"
# }}

# Voeg hierbij nooit "```json" aan toe.

# Zorg ervoor dat de output strikt voldoet aan dit JSON-formaat, zonder extra witruimte of aanvullende tekst.
# """

# Tag Call Queue

In [493]:
# call_queue = call_llm_service_queue(
#     summary, customer_comment, service, telesales, techniek, activatie, tag_prompt
# )
# print(f"Tag prompt output: {call_queue}")

In [494]:
call_queues = call_llm_service_queue(
    summaries, customer_comment, service, telesales, techniek, activatie, tag_prompt
)

for session_id, queue in call_queues.items():
    print(f"Session ID: {session_id}")
    print(f"Summary: {summaries[session_id]}")
    print(f"Customer Comment: {customer_comment}")
    print(f"Tagged queue: {queue}")
    print("-" * 50)

Data prepared: {'custom': {'session_id': '00001a3e-6022-4d94-a506-143856c60b0e', 'text': 'Je vraagt naar de status van je bestelling en hebt andere vragen over producten of diensten.', 'customer_comment': 'ik wil contact met een mens', 'service': "\nVoorbeelden: \n- Hulp bij vragen over facturen en betalingen\n- Hulp bij betaalproblemen en het treffen van betalingsregelingen\n- Uitleg over extra kosten of inflatiecorrectie\n- Hulp bij pin- en puk-code problemen\n- Informatie over huidige abonnement\n- Hulp bij gebruik van Mijn Odido en de app\n- Hulp bij inlogproblemen \n- Hulp bij abonnement koppelen\n- Wijzigen van persoonlijke of betaalgegevens\n- Hulp bij contractsovername\n- Aanvragen van (vervangende) sim- en esimkaarten\n- Hulp bij nummerportering\n- Hulp bij verhuizing van diensten\n- Uitleg over dienstgebruik, tarieven en roaming\n- Wijzigen van abonnementsextra's en bundels\n- Informatie over bestellingen, leveringen en retourneren\n- Wijzigen van instellingen (bijvoorbeeld v

In [495]:
# Create a list to store the data for each row
data = []

for session_id, queue in call_queues.items():
     # Parse the queue string into a dictionary
    try:
        queue_dict = json.loads(queue)
    except json.JSONDecodeError:
        # If JSON parsing fails, try eval (be cautious with this)
        try:
            queue_dict = eval(queue)
        except:
            print(f"Error parsing queue for session {session_id}: {queue}")
            queue_dict = {}
    
    # Create a dictionary for each row
    row = {
        "Session ID": session_id,
        "Summary": summaries[session_id],
        "Customer Comment": customer_comment,
        "Queue": queue_dict.get("queue", ""),
        "Queue Reason": queue_dict.get("reason", "")
    }
    data.append(row)

# Create a DataFrame
tagged_queue_df = pd.DataFrame(data)

# Display the DataFrame
print(tagged_queue_df)

Error parsing queue for session 0000b2f6-2244-4995-9e08-bf19def08bfd: ```json
{
    "queue": "Telesales",
    "reason": "Zoekt informatie over tv en internet, wat duidt op interesse in een (nieuw) abonnement."
}
```
                              Session ID                                                                                       Summary             Customer Comment      Queue                                                                      Queue Reason
0   00001a3e-6022-4d94-a506-143856c60b0e  Je vraagt naar de status van je bestelling en hebt andere vragen over producten of diensten.  ik wil contact met een mens    Service                          Vragen over status van bestelling en producten/diensten.
1   00000bdf-17cf-429e-aba9-bf5cfe4f9ba6                Je hebt geen wifi meer en zoekt naar een oplossing voor je internetverbinding.  ik wil contact met een mens   Techniek  Probleem met internetverbinding (geen wifi) valt onder technische ondersteuning.
2   00001339-

## Combine Dataframes

In [496]:
# Merge the two DataFrames on 'Session ID'
combined_df = pd.merge(conversation_summary_df, tagged_queue_df, on='Session ID', suffixes=('', '_tagged'))

# Select and rename the columns we want
final_df = combined_df[['Session ID', 'Conversation', 'Summary', 'Summary Character Count', 'Customer Comment', 'Queue', 'Queue Reason']]

# Optionally, reorder the columns if needed
final_df = final_df[['Session ID', 'Conversation', 'Summary', 'Customer Comment', 'Queue', 'Queue Reason', 'Summary Character Count']]

# Display the DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
print(final_df)

                              Session ID                                                                                         Conversation                                                                                       Summary             Customer Comment      Queue                                                                      Queue Reason  Summary Character Count
0   00001a3e-6022-4d94-a506-143856c60b0e  Bot: Hallo, ik ben chatbot Izzi van Odido.\nBot: Over welk product wil je iets vragen?\n\nUser b...  Je vraagt naar de status van je bestelling en hebt andere vragen over producten of diensten.  ik wil contact met een mens    Service                          Vragen over status van bestelling en producten/diensten.                       92
1   00000bdf-17cf-429e-aba9-bf5cfe4f9ba6  User: Ik heb geen wifi meer\n\nBot: Ik begrijp dat je een vraag hebt over Internet (+ TV + Vast ...                Je hebt geen wifi meer en zoekt naar een oplossing voor je internetverbinding

## Export Tagged Call Queue to Excel

In [497]:
# Get current date and time, format as YYYY-MM-DD_HH.MM
timestamp = datetime.now(ZoneInfo("Europe/Amsterdam")).strftime("%Y-%m-%d_%H.%M")

# Define output folder and filename
output_folder = "output"
filename = f"bot2call_summaries_call_queues_{timestamp}.xlsx"
full_path = os.path.join(output_folder, filename)

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(full_path, engine='xlsxwriter') as writer:
    # Write the dataframe to the Excel file
    final_df.to_excel(writer, sheet_name='Sheet1', index=False)
    
    # Get the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # Define a format for wrapped text
    wrap_format = workbook.add_format({'text_wrap': True})
    
    # Set column widths and apply text wrapping where needed
    column_settings = [
        ('A', 20),           # Session ID
        ('B', 60, True),     # Conversation (wrapped)
        ('C', 40, True),     # Summary (wrapped)
        ('D', 30, True),     # Customer Comment (wrapped)
        ('E', 15),           # Queue
        ('F', 40, True),     # Queue Reason (wrapped)
        ('G', 15)            # Summary Character Count
    ]
    
    for col, width, *wrap in column_settings:
        if wrap:
            worksheet.set_column(f'{col}:{col}', width, wrap_format)
        else:
            worksheet.set_column(f'{col}:{col}', width)

print(f"File successfully saved as: {filename}")

File successfully saved as: bot2call_summaries_call_queues_2025-08-07_16.53.xlsx
